# Generate list of submissions and survey form for Qualtrics

Make sure to save your Microsoft Forms responses in format "CSV UTF-8" with filename ``submissions.csv`` before running this script. If you've changed the text of the recommended form you'll need to update some of the strings below.

Packages needed for this notebook: ``jinja2`` and ``notebook``, e.g. run ``pip install jinja2 notebook``.

In [1]:
# Import a few standard packages
import csv
import jinja2
from IPython.core.display import HTML

In [23]:
# Read the submissions and do a bit of data cleaning
with open('submissions.csv', newline='', encoding='utf8') as csvfile:
    csvreader = csv.DictReader(csvfile, dialect='excel')
    submissions = list(csvreader)
for submission in submissions:
    for k, v in list(submission.items()):
        # This character shows up for some reason (unclear)
        k_new = k.replace('\ufeff', '').strip().split('\n')[0]
        # Some people use all caps for the title, so we automatically fix that
        if k_new=='Presentation title' and v.upper()==v:
            v = v.title()
        # some people paste text with newlines for every line which looks ugly, so we detect that and automatically fix
        if k_new=='Abstract (please keep under 300 words)' and max(map(len, v.split('\n')))<120:
            v = v.replace('\n', ' ')
        submission[k_new] = v
    submission['ID'] = submission['\ufeffID'] # not sure why forms inserts this random character

In [ ]:
# Just take a look at one example submission to make sure everything seems ok
submissions[0]

In [ ]:
# This generates the survey text that can be imported into Qualtrics

template = jinja2.Template('''
[[AdvancedFormat]]
    
{% for sub in submissions %}

[[Block]]

[[Question:DB]]
    
<div>
    <h3>{{ sub['Presentation title'] }}</h3>
    {% for para in sub['Abstract (please keep under 300 words)'].splitlines() %}
    {% if para.strip() %}
    <p style="font-size: 80%;">
        {{ para }}
    </p>
    {% endif %}
    {% endfor %}
</div>

[[Question:MC:SingleAnswer]]
[[ID:abstract{{ sub.ID }}yesno]]
I would like to see this as a talk.
[[Choices]]
No
Yes

[[Question:TextEntry:Form]]
[[ID:abstract{{ sub.ID }}comments]
Any comments?
[[Choices]]
Insert comments here.
{% endfor %}
''')

survey = template.render(submissions=submissions)

#HTML(template.render(submissions=submissions))
print(survey)
open('survey.txt', 'w', encoding='utf-8').write(survey)

In [ ]:
# This generates HTML to display all the submissions and saves to raw_submissions.html

template = jinja2.Template('''
<html><head><title>Submissions</title></head><body>
    
    <h1>All submissions</h1>
    {% for sub in submissions %}
    <div style="border: 1px solid grey; margin: 1em; padding: 1em;" >
        <h3>{{ sub['Presentation title'] }}</h3>
        <h4>{{ sub['Presentation authors'] }}</h4>
        <h4>Corresponding author: <a href="mailto:{{ sub['Corresponding author email address'] }}">{{ sub['Corresponding author name'] }}</a></h4>
        {% for para in sub['Abstract (please keep under 300 words)'].splitlines() %}
            {% if para.strip() %}
                <p>
                    {{ para }}
                </p>
            {% endif %}
        {% endfor %}
    </div>
    {% endfor %}
</body></html>
''')

submissions_html = template.render(submissions=submissions, int=int)

open('raw_submissions.html', 'w', encoding='utf-8').write(submissions_html)

HTML(submissions_html)

In [ ]:
max(list(map(len, submissions[-1]['Abstract (please keep under 300 words)'].split('\n'))))